In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from astropy.visualization import astropy_mpl_style
plt.style.use(astropy_mpl_style)
from astropy.utils.data import get_pkg_data_filename
from astropy.io import fits
from matplotlib.colors import LogNorm
import matplotlib.cm as cm
import pylab as pl
from astropy.table import QTable
from astropy.table import Table
import scipy.ndimage as ndi
from astropy.visualization import simple_norm
from astropy.modeling import models
from astropy.convolution import convolve
import photutils
import time
import statmorph
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#filter arrays
filters = np.array([277,356,444])
filter_names = ['f277w','f356w','f444w']

#dictionaries for the 3 primer files used to analyse morphology
all_primer_data = {}
all_primer_err_data = {}

#adding all the files to the data dictionary
for i in range(3):
    primer_file = get_pkg_data_filename(f'D:\primer_cosmos_nircam_v0.5_{filter_names[i]}_30mas_sci.fits')
    all_primer_data[i] = fits.getdata(primer_file, ext=0)

#adding all the error files to the data dictionary
for i in range(3):
    err_file = get_pkg_data_filename(f'D:\primer_cosmos_nircam_v0.5_{filter_names[i]}_30mas_err.fits')
    all_primer_err_data[i] = fits.getdata(err_file, ext=0)

In [ ]:
#creating function that can analyse galaxy morphology
#inputs are galaxy coordinates, filter and galaxy image size
def stat_morph(RA_coor,DEC_coor,Filter):
#Checking which filter should be used
    filts = np.array([277,356,444])
    filt = np.where(filts==Filter)[0][0]
    
    panel_size = 100
        
#finding the galaxy in the primer data fits file
    RA_val = find_nearest(RA_values, RA_coor)
    DEC_val = find_nearest(DEC_values, DEC_coor)
    RA_pixel = np.where(RA_values == RA_val)
    DEC_pixel = np.where(DEC_values == DEC_val)

#pixel ranges of galaxy image
    RA_min = RA_pixel[0][0] - panel_size/2
    RA_max = RA_pixel[0][0] + panel_size/2
    DEC_min = DEC_pixel[0][0] - panel_size/2
    DEC_max = DEC_pixel[0][0] + panel_size/2
    
#defining area with galaxy
    galaxy_image = all_primer_data[filt][DEC_min:DEC_max,RA_min:RA_max]
    galaxy_err = all_primer_err_data[filt][DEC_min:DEC_max,RA_min:RA_max]


#creating segmentation map
    threshold = photutils.detect_threshold(galaxy_image, 3)
    npixels = 2  # minimum number of connected pixels
    segm = photutils.detect_sources(galaxy_image, threshold, npixels)
    label = np.argmax(segm.areas) + 1
    segmap = segm.data == label
#and making it smoother
    segmap_float = ndi.uniform_filter(np.float64(segmap), size=10)
    segmap = np.int64(segmap_float > 0.5)
    


#performing the statmorph calculations
    source_morphs = statmorph.source_morphology(
        galaxy_image, segmap, weightmap=galaxy_err)
    morph = source_morphs[0]
    
    return morph, galaxy_image, segmap

In [ ]:
#reading the file with different emitters
Halpha_emitters = np.loadtxt('H-alpha emitters')
Hbeta_O3_emitters = np.loadtxt('H-beta and O3 emitters')
O2_emitters = np.loadtxt('O2 emitters')

In [ ]:
#choose emission line group (possible inputs: Halpha_emitters, Hbeta_O3_emitters, O2_emitters)
emitters = ?

#Choose filter corresponding to chosen emission line group base upon morphological k-correction (Eq. 4.3) (possible inputs: 277, 356, 444)
Filter = ?

#choose which galaxy to plot (can range from 0 to amount of galaxies within above defined emission line group)
x = ?

RA_coor = emitters[0][x]
DEC_coor = emitters[1][x]

morph = stat_morph(RA_coor, DEC_coor, Filter)

print('C =', morph[0].concentration)
print('A =', morph[0].asymmetry)
print('S =', morph[0].smoothness)
print('$r_{Petro}$ =', morph[0].rpetro_ellip)